# TP 2 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [55]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=Int64[]
    weight=Int64[]
    KnapCap=Int64[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end


readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [2]:
function testSondability_LP(faisable, BorneSup, listxi, BestProfit, Bestsol, bool)
    TA, TO, TR = false, false, false
    boolean = false
    for i in listxi
        if i==1
            boolean = true
            break
        end
    end

    if(!faisable)#Test de faisabilite
        TA=true
        println("TA")
    
    elseif( prod(abs.([round.(v, digits=0) for v in listxi]-listxi) .<= fill(10^-5, size(listxi))) || bool || boolean
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (BorneSup >= BestProfit)
            Bestsol = copy(listxi)
            #BestProfit=value(benef)
            BestProfit=BorneSup
            println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    elseif(BorneSup <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

testSondability_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [3]:
function separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, n, obj)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left

    # lexicographic branching criterion: branch on the 1st object not yet fixed
    
    #= i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end =#
    

    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end


function exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible

    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop
end

exploreNextNode_depthfirst! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [4]:
function borne1(capacity, weight, price, listVals, listObjs, listxi)

    borneSup = 0
    borneInf = 0
    capacityRest = capacity
    cout = 0
    ratioMax = 0
    objRest = [] 
    ratio = Dict()
    n = length(listxi)
    ident = 1
    obj = 1
    obj2 = 0

    for i in listObjs
        capacityRest -= weight[i] * listVals[ident]

        if capacityRest < 0 
            obj2 = i
        end
        cout += price[i]*listVals[ident]
        ident += 1
        
    end 

    for i in 1:n
        if(!(i in listObjs))
            push!(objRest,i)
            ratio[price[i]/weight[i]] = i
        end       
    end


    if (length(objRest) != 0)
        ratioMax = maximum(ratio)[1]
        obj = ratio[ratioMax]
        if (capacityRest >=0)
            listxi[obj] = capacityRest/weight[obj]
        end
    else
        ratioMax = price[listObjs[n]]/weight[listObjs[n]]
    end

    borneSup = capacityRest*ratioMax + cout


    return listxi, borneSup, capacityRest, obj, obj2
end

borne1 (generic function with 1 method)

In [15]:
function borne2(capacity, price, weight, listobjs, listvals, listxi)
    capacityRest = capacity
    borneSup = 0
    k = 1
    r = zeros(length(price))
    obj = 0
    for i in 1:length(price)
        r[i] = price[i] / weight[i]
    end
    r_trie = sortperm(r, rev = true)

    for i in 1:length(listobjs)
        j = listobjs[i]
        listxi[j] = listvals[i]
        borneSup += listvals[i]*price[j]
        capacityRest -= listvals[i]*weight[j] 
    end 

    while capacityRest > 0 && (k<= length(price))
        i = r_trie[k]
        if !(i in listobjs)
            if (capacityRest < weight[i])
                listxi[i] = 0
                capacityRest = capacityRest - listxi[i]*weight[i]
                break
            else
                listxi[i] = 1
                capacityRest = capacityRest - weight[i]
            end
            borneSup += listxi[i]*price[i]
            obj = i          
        end
        k+=1        
    end   
    return listxi,borneSup, obj
end

borne2 (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [22]:
function solveKnapInstance(filename, BS)

    price, weight, capacity = readKnaptxtInstance(filename)


    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=Int64[]
    listvals=Float64[]
    listnodes=Int64[]

    BestProfit::Float64=-1.0
    Bestsol=Float64[]

    current_node_number::Int64=0
    stop = false
    n = length(price)
    listxi = zeros(n, 1)
    faisable = true
    BorneSup = 0
    SolCourante = zeros(n,1)
    capacityRest = capacity
    obj = 0
    obj2 = 0
    bool = false


    while(!stop)
        
        println("\nNode number ", current_node_number, ": \n---------------\n")
        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        if BS == "BORNE1"

            listxi, BorneSup, capacityRest, obj, obj2 = borne1(capacity, weight, price, listvals, listobjs, listxi)
            if (capacityRest < 0)
                faisable = false
                capacityRest += weight[obj2]
            end

        elseif BS == "BORNE2"
            listxi, BorneSup, obj = borne2(capacityRest,price, weight, listobjs, listvals, listxi)
        else
            print("UNFOUND")
        end
        
        println("\nBorneSup ", BorneSup, "\n------")
                
        #create LP of current node
        print("Solve the LP model of the current node to compute its bound: start ... ")

        println("... end"); 

        print(": Solution LP")
        if(!faisable)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", BorneSup)
            [print("\t", "x[", i, "]=", listxi[i]) for i in 1:n] 
        end
        println(" "); 

        println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        if length(listobjs) == length(price)
            bool = true
        end
        
        TA, TO, TR, Bestsol, BestProfit = testSondability_LP(faisable, BorneSup, listxi, BestProfit, Bestsol, bool)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            separateNodeThenchooseNext_lexicographic_depthfirst!(listobjs, listvals, length(price), obj)
        else
            stop = exploreNextNode_depthfirst!(listobjs, listvals, listnodes)
        end
        current_node_number = current_node_number + 1

        stop = current_node_number > 10

        faisable = true
        bool = false
    end
    #println("\n******\n", listxi)
    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


solveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [23]:
function solveNdisplayKnap(filename)

    println("\n Branch-and-Bound for solving a knapsack problem. \n\n Solving instance '" * filename * "'\n")

    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = solveKnapInstance(filename, "BORNE1")

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    println("\n Branch-and-bound tree visualization : start display ...")
    display(graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree))
    println("... end display. \n\n")

end

solveNdisplayKnap (generic function with 1 method)

In [25]:
INSTANCE = "instances/test.opb.txt"

solveNdisplayKnap(INSTANCE)

println("press enter to exit ! ")
readline()


 Branch-and-Bound for solving a knapsack problem. 

 Solving instance 'instances/test.opb.txt'


Node number 0: 
---------------


BorneSup 16.666666666666668
------
Solve the LP model of the current node to compute its bound: start ... ... end
: Solution LP 16.666666666666668	x[1]=0.0	x[2]=1.6666666666666667	x[3]=0.0	x[4]=0.0 

Previous Solution memorized Float64[] with bestprofit -1.0

non sondable

branch on object 2


Node number 1: 
---------------


BorneSup 16.4
------
Solve the LP model of the current node to compute its bound: start ... ... end
: Solution LP 16.4	x[1]=0.0	x[2]=1.6666666666666667	x[3]=0.8	x[4]=0.0 

Previous Solution memorized Float64[] with bestprofit -1.0

non sondable

branch on object 3


Node number 2: 
---------------


BorneSup 16.8
------
Solve the LP model of the current node to compute its bound: start ... ... end
: Solution LP : NOT AVAILABLE (probably infeasible or ressources limit reached) 

Previous Solution memorized Float64[] with bestprofit -1

LoadError: UndefVarError: `graphplot` not defined

**Questions préliminaires**

**1. Quelle est la règle de séparation choisie ?**

La règle de séparation choisie dans la fonction separateNodeThenchooseNext_lexicographic_depthfirst! est le critère lexicographique. La fonction parcourt les objets non encore fixés dans l'ordre de leur indice (de 1 à n), et elle sélectionne le premier objet non fixé comme critère de branchement. Cela signifie qu'elle choisit le premier objet qui n'a pas encore été pris en compte dans la solution partielle actuelle du problème du sac à dos.

**2. Quelle méthode de calcul de borne supérieure est utilisée ?**

La méthode de calcul de borne supérieure est déterminée par la résolution d'une relaxation linéaire, cette relaxation linéaire est résolue à chaque nœud de l'arbre de recherche en utilisant un modèle linéaire (LP) du sac à dos où x appartient à IR et x entre 0 et 1.

**3. Quels sont les tests de sondabilité TA, TO, TR ?**

TA  :  Le test d'admissibilité vérifie si la capacité restante, après ajout des objets sélectionnés, est négative. 
TO  :  Le test d'optimalité compare la borne supérieure du nœud à la meilleure solution connue. 
TR  :  Le test de résolution s'assure de la faisabilité de la solution du nœud. En cas de faisabilité, il vérifie si la valeur de la fonction objective du nœud est meilleure que la meilleure valeur de profit initialement considérée, si c'est le cas, la solution optimale est mise à jour

**4. Quelle est la stratégie d’exploration choisie ?**

La stratégie d'exploration choisie dans le code est une stratégie de parcours en profondeur (depth-first) dans l'arbre de recherche. Cette stratégie consiste à explorer aussi loin que possible le premier sous-arbre avant de revenir en arrière et d'explorer d'autres sous-arbres.

**Code et analyse**

**4. Les points clés de notre implémentation des différents blocs du Branch-and-Bound :**

En premier lieu, la fonction testSondability_LP a été adaptée pour fonctionner indépendamment de toute référence à un modèle d'optimisation spécifique. Au lieu de cela, elle prend des paramètres tels que faisable, BorneSup, listxi, BestProfit, Bestsol, et bool.
'faisable' : Remplace la fonction 'termination_status'. Il est vrai si la capacité restante est positive, et fausse sinon, dans ce cas TA est réalisé.
'listxi' : La liste des quantités attribuées à chaque objets. Dans chaque noeud de l'arbre la liste sera mis à jour.
Ensuite, on a ajouté une boucle pour parcourir listxi et vérifier s'il existe une valeur réalisable (proche de 1). Cela introduit une nouvelle logique de vérification qui n'était pas présente dans le code initial pour le test de sondabilité TR.

Pour la fonction 'separateNodeThenchooseNext_lexicographic_depthfirst', on a ajouté un paramétre 'obj' qui gére l'objet qu'on veut ajouter à la liste des objets choisis.

La borne 1 utilise la relaxation continue, permettant des valeurs fractionnaires, tandis que la borne 2 utilise la relaxation entière, permettant uniquement des valeurs entières pour les variables de décision. 

Borne 1: 

Utilisation d'une liste (listxi) pour stocker les fractions d'objets sélectionnées.
La fonction parcourt la liste listObjs (qui contient les indices des objets sélectionnés) et met à jour la capacityRest en soustrayant les poids des objets sélectionnés multipliés par leurs fractions associées.
Si la capacité restante devient négative lors de la mise à jour, la variable obj2 est mise à jour avec l'indice de l'objet qui a dépassé la capacité.
La variable cout est mise à jour en additionnant les prix des objets sélectionnés multipliés par la listvals.
Une liste objRest est créée pour stocker les indices des objets non sélectionnés.
Un dictionnaire ratio est utilisé pour stocker les ratios prix/poids des objets non encore sélectionnés.
Si la liste objRest n'est pas vide, le ratio maximum parmi les objets non sélectionnés est calculé.
Enfin, on calcul la borne supérieure en utilisant la formule, capacité restante * ratio maximal, et on retourne la borne calculée, la liste listxi et l'objet courant.


Borne 2: 

Initialisation du vecteur des ratios prix/poids.
Mise à jour des fractions et de la capacité restante.
Si la capacité restante est insuffisante pour inclure un objet , on met 0 mais on calcule la fraction de l'objet suffisante pour atteindre la capacité maxiamle
La borne supérieure (borneSup) est mise à jour en tenant compte des fractions des objets sélectionnés.
Enfin, on retourne la liste des fractions, la nouvelle borne supérieure, et l'identifiant de l'objet sélectionné.

Dans la fonction 'solveKnapInstance', on a ajouté un paramétre BS qui sert à identifié si on veut travailler avec la borne 1 ou la borne2. Dans le cas ou on choisi 'BORNE1', on teste toujours si la capacité restante et positive pour signaler le test TA sinon.

**5. Quelle est la règle de séparation choisie ?**

On a commencé par un test de l'exemple vu en cours/tp (test.opb.txt) pour valider le fonctionnement de notre modèle de la borne1/borne2. La borne 1 donne exactement
le resultat vu en cours. Ensuite, On a créé un test particulier pour la borne 2 dont on connait le résultat (test2.opb.txt), les 4 premières itérations donnent les résultats attendus, tandis que la suite des noeuds ne se mettent pas tous à jour.

La borne 1 fonctionne parfaitement avec les autres tests notamment les tests qu'on a mis dans le dossier instances.

**6. Solutions d'amélioration de Branch and Bound :**

Utiliser une séparation plus sophistiquée pour choisir quels objets diviser en deux sous-noeuds. Par exemple, choisir des objets basés sur leur impact sur la borne supérieure.
Intégrer une relaxation lagrangienne pour améliorer la qualité des bornes supérieures.
Modifier la stratégie pour choisir dynamiquement la prochaine variable à fixer en fonction de son impact sur la résolution.
Mettre en œuvre des stratégies de parallélisation pour explorer simultanément plusieurs branches de l'arbre de recherche.